# Section 3.7 – Travaux pratiques

Ce notebook regroupe des exemples R autour de la **méthode du bootstrap** et de la simulation de variables aléatoires. Les commentaires détaillent chaque étape pour que vous puissiez suivre facilement.


## 3.7.1 Méthode du bootstrap et simulation de lancer de dé

Commençons par créer une fonction $lancer_le_de(n)$ qui simule $n$ lancers d’un dé équilibré (valeurs 1 à 6).


In [ ]:

# Fonction pour simuler n lancers d'un dé (équilibré)
lancer_le_de <- function(n) {
  # sample() tire des valeurs dans 1:6 avec remise
  sample(1:6, n, replace = TRUE)
}

# Exemple : 20 lancers
set.seed(10)
res <- lancer_le_de(20)
res

# Estimation de la probabilité d'obtenir un 4 dans cet échantillon
p_hat <- sum(res == 4) / length(res)
p_hat


Lorsque l'on répète le lancer de 20 dés, la proportion de $4$ varie d'un échantillon à l'autre. Avec un nombre très grand de lancers ($n = 10000$), la fluctuation est beaucoup plus faible.


In [ ]:

# Comparaison avec un grand nombre de lancers
n <- 10000
res_large <- lancer_le_de(n)
prop1 <- sum(res_large == 4) / n

# Un autre tirage
res_large2 <- lancer_le_de(n)
prop2 <- sum(res_large2 == 4) / n

c(prop1 = prop1, prop2 = prop2)


On constate que les proportions se rapprochent de la valeur théorique $p = 1/6 pprox 0{,}1667$.  
Nous allons maintenant estimer numériquement le **biais** et la **variance** de l'estimateur $\hat{p}$ (la fréquence de $4$) par simulation de Monte‑Carlo.


In [ ]:

# Monte‑Carlo pour estimer le biais et la variance de l'estimateur p_hat
n <- 20           # taille de l'échantillon
M <- 10000        # nombre de répétitions

# Répéter M fois : simuler n lancers et calculer la fréquence de 4
set.seed(123)
vec_p_hat <- replicate(M, {
  res <- lancer_le_de(n)
  sum(res == 4) / n
})

# Estimations du biais et de la variance
biais_estime <- mean(vec_p_hat) - (1/6)
variance_estime <- var(vec_p_hat)

c(biais = biais_estime, variance = variance_estime)

# Comparaison avec la formule théorique p(1 - p)/n
p <- 1/6
variance_theorique <- p * (1 - p) / n
variance_theorique


L'estimateur de proportion est sans biais et sa variance théorique vaut $p(1-p)/n$. Les valeurs simulées confirment ces résultats.

### Méthode du bootstrap
Le bootstrap consiste à estimer le biais et la variance à partir d'un seul échantillon en resélectionnant avec remise.


In [ ]:

# Bootstrap à partir d'un seul échantillon de 20 lancers
n <- 20
set.seed(456)
xvec <- lancer_le_de(n)

# Proportion observée de 4
p_hat_obs <- sum(xvec == 4) / n
p_hat_obs

# Nombre de rééchantillonnages bootstrap
B <- 10000

# Rééchantillonnage avec remise et calcul de la proportion de 4 à chaque fois
vec_p_star <- replicate(B, {
  resample <- sample(xvec, n, replace = TRUE)
  sum(resample == 4) / n
})

# Estimation du biais bootstrap
biais_bootstrap <- mean(vec_p_star) - p_hat_obs

# Variance bootstrap (corrigée de B/(B-1))
variance_bootstrap <- ((B - 1) / B) * var(vec_p_star)

c(biais_bootstrap = biais_bootstrap, variance_bootstrap = variance_bootstrap)


Le bootstrap donne une estimation du biais et de la variance proches des valeurs théoriques à partir d’un seul échantillon.  

## 3.7.2 Simulation de lois
Il arrive que l’on doive simuler des variables aléatoires dont la loi n’est pas directement disponible dans R. Voici deux méthodes classiques : la **méthode de la transformation inverse** et la **méthode de simulation discrète**.


### Méthode de la transformation inverse pour une loi exponentielle

On sait que si $X\sim	ext{Exp}(\lambda)$ alors sa fonction de répartition est
$$
F_X(x) = 1 - e^{-\lambda x} \quad (x \ge 0)
$$  
L’inverse de cette fonction est :
$$
F_X^{-1}(u) = -rac{1}{\lambda}\,\ln(1 - u).
$$
On peut donc simuler $n$ observations de $Exp(2)$ en utilisant des réalisations $u$ issues d’une loi uniforme sur $[0,1]$.


In [ ]:

# Simulation de n variables exponentielles Exp(2) par transformation inverse
n <- 10000
lambda <- 2

# Générer des U uniformes
u <- runif(n)

# Appliquer l'inverse de la fonction de répartition
X_exp <- - (1 / lambda) * log(1 - u)

# Vérification visuelle : histogramme et densité théorique
hist(X_exp, breaks = 30, freq = FALSE, col = "lightgreen",
     main = "Simulation de Exp(2) par transformation inverse", xlab = "x")
curve(dexp(x, rate = lambda), add = TRUE, col = "red", lwd = 2)


Les deux courbes se superposent, confirmant que la simulation correspond bien à une loi exponentielle de paramètre 2.


### Simulation d’une variable discrète uniforme sur {0,1,2,3,4,5}

On peut simuler une variable discrète uniforme en divisant l’intervalle $[0,1]$ en six sous‑intervalles de longueurs égales et en affectant une valeur entière selon l’endroit où tombe un nombre uniforme $U$. Une façon simple consiste à utiliser la fonction $sample()$, mais voici un algorithme illustrant la méthode expliquée dans le polycopié.


In [ ]:

# Simulation d'une variable uniforme discrète sur {0,1,...,5} via runif()

n <- 20

# Génération des U uniformes
U <- runif(n)

# Fonction pour transformer U en variable discrète
sim_discrete <- function(u) {
  # Il y a 6 intervalles de longueur 1/6
  return(floor(u * 6))
}

# Application
X_disc <- sapply(U, sim_discrete)
X_disc

# Vérification des fréquences obtenues
barplot(table(X_disc), col = "orange", main = "Simulation uniforme discrète", xlab = "valeur")


Ce code découpe l’intervalle $[0,1]$ en six parties égales. La fonction $floor(u * 6)$ produit un entier entre 0 et 5, chaque valeur ayant la même probabilité.

## Conclusion
Ces exemples montrent comment utiliser R pour simuler des données, estimer des paramètres et appliquer la méthode du bootstrap. N’hésitez pas à adapter les tailles d’échantillons et le nombre de répétitions ($M$ ou $B$) selon la précision souhaitée.
